In [1]:
from hydra.experimental import compose, initialize
import hydra
from copy import copy
from omegaconf import OmegaConf
from box import Box, BoxList
import os
from datetime import datetime
from itertools import product
from pprint import pprint

logs = BoxList([])
seed = 78

import random
import numpy as np
np.random.seed(seed)
random.seed(seed)
from pyleaves.utils import ensure_dir_exists

# overrides = ['dataset@dataset=PNAS_family_100', 'model@model=resnet_50_v2', 'model.params.optimizer="SGD"', 'model.params.weights="imagenet"', 'model.params.lr=1e-5', 'model.params.regularization.l2=1e-3', 'callbacks.early_stopping.patience=3', 'tags=["PNAS_family_100","debug","env=pyleaves2.2"]', 'dataset.params.training.target_size=[512,512]', 'dataset.params.training.batch_size=16', 'model.params.head_layers=[512,256]', 'dataset.params.training.num_epochs=30', 'orchestration.debug=True']#,  '--multirun']

def main(config_name="simplified_config", overrides: list=[]):

    with initialize(config_path="configs"):
        config = compose(config_name=config_name, overrides=overrides)

    return config
# config = main(overrides=overrides)

# config = main(overrides=['dataset@dataset=Leaves_family_100'])
# config = main(overrides=['dataset@dataset=PNAS_family_100'])

# initialize(config_path="configs")
# config_name="simplified_config"
# # config_0 = compose(config_name=config_name, overrides=overrides)

# overrides=['dataset@dataset=PNAS_family_100']
# config = compose(config_name=config_name, overrides=overrides)

# test_config = main(overrides=['dataset@dataset=Fossil_family_100_test',
#                               '~dataset.params.training.target_size',
#                              'dataset.params.training.augmentations=[]'])
# # merged_config = OmegaConf.merge(config, test_config)
# # copied = OmegaConf.masked_copy(test_config, ["dataset"])
# # pprint(OmegaConf.to_container(copied))
# # pprint(OmegaConf.to_container(test_config))

# def resolve_test_config(main_config, test_data_config):

#     copied_test_cfg = OmegaConf.masked_copy(test_data_config, ["dataset"])
#     resolved_main_cfg = OmegaConf.to_container(main_config, resolve=True)
#     resolved_main_cfg = OmegaConf.create(resolved_main_cfg)
#     merged_config = OmegaConf.merge(resolved_main_cfg, copied_test_cfg)
    
#     return merged_config

# config.dataset.params.training

# config.task = 0
# config.dataset.params.training.target_size = [768,768]
# merged_config = resolve_test_config(main_config=config, test_data_config=test_config)

# # merged_config.task = 0
# pprint(OmegaConf.to_container(merged_config, resolve=True))

# type(config)

# Programmatically transferring images into class-specific hierarchical directories

### Transforming a 2-level hierarchy into single level in preparation for using an ImageDataGenerator from tf.keras



We have a dataset of images from 3 different collections (i.e. original publishing source) with overlapping subsets of class labels in the form of biological family (e.g. Theaceae, Rosaceae).

## source structure:
```
./wolfe/
        Acanthaceae/
            image_0.jpg
            image_1.jpg
            ...
        Achariaceae/
        ...
        Winteraceae/
        Zygophyllaceae/
./axelrod/
        Acanthaceae/
            ...
        ...
./klucking/
        ...
```


## target structure:

```
Acanthaceae/
    image_0.jpg
    image_1.jpg
    ...
Achariaceae/
...
Winteraceae/
Zygophyllaceae/



```



In [2]:
import os
import shutil
# PNAS_dir = '/media/data_cifs/jacob/Fossil_Project/data/scratch_data/PNAS 2016 leaves'#/PNAS_family'
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
from pathlib import Path

PNAS_dir = '/media/data_cifs_lrs/projects/prj_fossils/data/processed_data/PNAS_2020-06'
output_root_dir = '/media/data_cifs_lrs/projects/prj_fossils/data/processed_data/PNAS_2020-06/PNAS_family'
collections = ['wolfe','axelrod','klucking']
collection_dirs = [os.path.join(PNAS_dir, collection) for collection in collections]
collection_dirs

def get_collection_files(collection_dir, position=0, leave=False):
    collection_files = {}
    for family in tqdm(os.listdir(collection_dir), desc=f'collection {Path(collection_dir).name}', position=position, leave=leave):
        family_dir = os.path.join(collection_dir, family)
        fam_files = [os.path.join(family_dir, file) for file in os.listdir(family_dir)]
        if family not in collection_files.keys():
            collection_files[family] = []
        collection_files[family].extend(fam_files)
        
    return collection_files
    
def gather_collections(collection_dirs, leave=False):
    all_files = {}
    for collection_dir in tqdm(collection_dirs, desc=f'PNAS collections', position=0, leave=leave):
        collection_files = get_collection_files(collection_dir, position=1, leave=leave)
        for family, fam_files in collection_files.items():
            if family not in all_files.keys():
                all_files[family] = []
            all_files[family].extend(fam_files)
    return all_files

all_files = gather_collections(collection_dirs, leave=True)

/media/data/conda/jacob/envs/pyleaves2.2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
leave=True
for family, family_files in  tqdm(all_files.items(), desc='Family directories', position=0, leave=leave):
    family_dir = os.path.join(output_root_dir, family)
    os.makedirs(family_dir)
    for file in tqdm(family_files, desc = f'{family} files', position=1, leave=False):
        shutil.copy(file, family_dir)


KeyboardInterrupt: 

################################################################################################################################################
################################################################################################################################################
# NEW SECTION
################################################################################################################################################
################################################################################################################################################

In [ ]:
# import tensorflow as tf
# from sklearn.metrics import classification_report
# import os
# import numpy as np
# # model_path = "/media/data/jacob/simplified-baselines/Leaves_in_PNAS_family_100_resnet_50_v2_[512, 512]/task-9_2020-09-22_23-25-32/model_dir/saved_model"
# # model = tf.keras.models.load_model(model_path)
# model_config.num_classes = encoder.num_classes
# model = build_model(model_config)

# # from hydra.experimental import compose, initialize
# from omegaconf import OmegaConf, DictConfig

# # with initialize(config_path="configs"):
# #     config = compose(config_name="config", overrides=['dataset@dataset=PNAS','use_tfrecords=False'])
# #     print(config.pretty())
# import hydra
# # from pyleaves.pipelines.pipeline_1 import *

# from pyleaves.datasets import base_dataset

# from pyleaves.utils.experiment_utils import resolve_config_interpolations
# from paleoai_data.utils.kfold_cross_validation import DataFold
# from typing import List, Union
# import random
# import numpy as np
# from more_itertools import unzip
# import shutil
# import os
# import neptune
# from pathlib import Path
# import yaml



In [2]:
def summarize_sample(x, y):
    y_int=y
    y_encoding = 'sparse int'
    if isinstance(y, np.ndarray):
        y_int = np.argmax(y, axis=-1)
        if y.ndim>=1 and y.shape[-1] > 1:
            y_encoding = 'one hot'
    print(f'y = {y_int} [{y_encoding} encoded]')
    print(f'y.dtype = {y.dtype}, x.dtype = {x.dtype}\n')
    print(f'y.shape = {y.shape},\ny.min() = {y.min():.3f} | y.max() = {y.max():.3f},\ny.mean() = {y.mean():.3f} | y.std() = {y.std():.3f}\n')
    print(f'x.shape = {x.shape},\nx.min() = {x.min():.3f} | x.max() = {x.max():.3f},\nx.mean() = {x.mean():.3f} | x.std() = {x.std():.3f}')

    plt.imshow(x)

# from pyleaves.pipelines.pipeline_simple import *
# from hydra.core.hydra_config import HydraConfig
from pyleaves.utils import set_tf_config
gpu = set_tf_config(gpu_num=None, num_gpus=1, wait=0)

import tensorflow as tf
from tensorflow.keras import backend as K
K.clear_session()

from pyleaves.utils.pipeline_utils import build_model
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from pprint import pprint
from box import Box
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

import neptune
# import neptune_tensorboard as neptune_tb

from neptunecontrib.monitoring.keras import NeptuneMonitor

# neptune_tb.integrate_with_tensorflow()





#     featurewise_center=False, samplewise_center=False,
#     featurewise_std_normalization=False, samplewise_std_normalization=False,
#     zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
#     height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
#     channel_shift_range=0.0, fill_mode='nearest', cval=0.0, horizontal_flip=False,
#     vertical_flip=False, rescale=1.0/255, preprocessing_function=preprocess_input,
#     data_format=None, validation_split=validation_split, dtype=np.uint8
# )





neptune_project_name = 'jacobarose/jupyter-testing-ground'

params = Box({
              'image_dir': '/media/data_cifs_lrs/projects/prj_fossils/data/processed_data/PNAS_2020-06/PNAS_family',
              'log_dir': '/media/data_cifs_lrs/projects/prj_fossils/users/jacob/tensorboard_log_dir',          
              'validation_split': 0.1,
              'target_size':(299,299),
              'batch_size':32,
              'num_epochs': 10,
              'seed': 20,
              'rescale': None, #1.0/255,
              'preprocess_input': "tensorflow.keras.applications.resnet_v2.preprocess_input", #None,
              'color_mode': 'rgb',
              'early_stopping': {'monitor':"val_loss",
                                'patience':10,
                                'min_delta':0.01,
                                'restore_best_weights':True}
    
})

# from omegaconf import OmegaConf
print(params.to_yaml())

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=params.rescale,
                                                          preprocessing_function=params.preprocess_input,
                                                          validation_split=params.validation_split)

train_data = datagen.flow_from_directory(
    params.image_dir, target_size=params.target_size, color_mode=params.color_mode, classes=None,
    class_mode='categorical', batch_size=params.batch_size, shuffle=True, seed=params.seed,
    subset='training', interpolation='nearest')

val_data = datagen.flow_from_directory(
    params.image_dir, target_size=params.target_size, color_mode=params.color_mode, classes=None,
    class_mode='categorical', batch_size=params.batch_size, shuffle=False, seed=params.seed,
    subset='validation', interpolation='nearest')

params.num_samples_train = train_data.samples
params.num_samples_val = val_data.samples
params.num_classes = train_data.num_classes
steps_per_epoch=params.num_samples_train//params.batch_size
validation_steps=params.num_samples_val//params.batch_size
model_config = Box({
                    'model_name': "resnet_50_v2",
                    'optimizer':"Adam",
                    'num_classes':params.num_classes,
                    'weights': "imagenet",
                    'frozen_layers':'bn', #None, #(0,-4),
                    'input_shape':(*params.target_size,3),
                    'lr':1e-5,
                    'lr_momentum':None,#0.9,
                    'regularization':{},#{"l2": 1e-4},
                    'loss':'categorical_crossentropy',
                    'METRICS':['f1','accuracy'],
                    'head_layers': [256,128]
                    })

neptune.init(project_qualified_name=neptune_project_name)



model = build_model(model_config)
model.summary()

# base = model.layers[0]
# dir([l for l in base.layers if l.name.endswith('bn')][0])
# def freeze_batchnorm_layers(model, verbose=False):
#     for i, layer in enumerate(model.layers):
#         if layer.name.endswith('bn'):
#             if verbose: print(f'layer {i}', layer.name)
#             layer.trainable = False
#     return model

# base = freeze_batchnorm_layers(base, verbose=True)

# len(base.layers)

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
callbacks = [TensorBoard(log_dir=params.log_dir, profile_batch=0),
             NeptuneMonitor(),
             EarlyStopping(monitor=params.early_stopping.monitor,
                           patience=params.early_stopping.patience,
                           min_delta=params.early_stopping.min_delta, 
                           verbose=1, 
                           restore_best_weights=params.early_stopping.restore_best_weights)]


neptune_params = {k: str(v) for k,v in params.items() if type(v)==dict}

with neptune.create_experiment(name=neptune_experiment_name, params=params):
    model.summary(print_fn=lambda x: neptune.log_text('model_summary', x))

    print('[BEGINNING TRAINING]')
    try:
        history = model.fit(train_data,
                            epochs=num_epochs,
                            callbacks=callbacks,
                            validation_data=val_data,
                            validation_freq=1,
                            shuffle=True,
                            steps_per_epoch=steps_per_epoch,
                            validation_steps=validation_steps,
                            verbose=1)

    except Exception as e:
        raise e

/media/data/conda/jacob/envs/pyleaves2.2/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


setGPU: Setting GPU to: [2]
Initial visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
visible GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Successfully set memory_growth=True and limited GPUs visible to tensorflow.

Now using GPU(s):
['/physical_device:GPU:0']
batch_size: 32
color_mode: rgb
early_stopping:
  min_delta: 0.01
  monitor: val_loss
  patience: 10
  restore_best_weights: true
image_dir: /media/data_cifs_lrs/projects/prj_fossils/data/processed_data/PNAS_2020-06/PNAS_family
log_dir: /media/data_cifs_lrs/projects/prj_fossils/users/jacob/tensorboard_log_dir
num_epochs: 10
preprocess_input: tensorflow.keras.applications.resnet_v2.preprocess_input
rescale: null
seed: 20
target_size: !!python/tuple
- 299
- 299
validation_split: 0.1

Found 456 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)               

ValueError: too many values to unpack (expected 2)

In [3]:
params

<Box: {'image_dir': '/media/data_cifs_lrs/projects/prj_fossils/data/processed_data/PNAS_2020-06/PNAS_family', 'log_dir': '/media/data_cifs_lrs/projects/prj_fossils/users/jacob/tensorboard_log_dir', 'validation_split': 0.1, 'target_size': (299, 299), 'batch_size': 32, 'num_epochs': 10, 'seed': 20, 'rescale': None, 'preprocess_input': 'tensorflow.keras.applications.resnet_v2.preprocess_input', 'color_mode': 'rgb', 'early_stopping': {'monitor': 'val_loss', 'patience': 10, 'min_delta': 0.01, 'restore_best_weights': True}, 'num_samples_train': 456, 'num_samples_val': 50, 'num_classes': 2}>

In [ ]:
    if os.path.exists(csv_path):
        neptune.log_artifact(csv_path)

    model.save(config.run_dirs.saved_model_path)
    print('[STAGE COMPLETED]')
    print(f'Saved trained model to {config.run_dirs.saved_model_path}')

    print('history.history.keys() =',history.history.keys())

    steps = split_datasets['test'].num_samples//data_config.training.batch_size

    test_results = evaluate(model, encoder, model_config, data_config, test_data=test_data, steps=steps, num_classes=encoder.num_classes, confusion_matrix=True)

    print('TEST RESULTS:')
    pprint(test_results)

    # for k,v in test_results.items():
    #     neptune.log_metric(k, v)
    # predictions = model.predict(test_data, steps=split_datasets['test'].num_samples)

print(['[FINISHED TRAINING AND TESTING]'])

# return test_results

In [54]:
# config.task = 0
# data_config = config.dataset.params
# extract_config = config.dataset.params.extract
# training_config = config.dataset.params.training
# model_config = config.model.params
# preprocess_config = config.model.params.preprocess_input
# model_config.input_shape = (*training_config.target_size, extract_config.num_channels)
# config.model.params = model_config

##############################################
# test_stage_config = init_Fossil_family_100_test_config(main_config=config)
# test_data_config = test_stage_config.dataset.params
# test_fold_dir = test_data_config.extract.fold_dir
# test_fold_id = test_data_config.extract.fold_id
# test_fold_path = DataFold.query_fold_dir(test_fold_dir, test_fold_id)
# fossil_test_fold = DataFold.from_artifact_path(test_fold_path)
##############################################

# fold_path = DataFold.query_fold_dir(extract_config.fold_dir, extract_config.fold_id)
# fold = DataFold.from_artifact_path(fold_path)
# ##############################################

# encoder = init_pipeline_encoder_scheme(fold, test_fold=fossil_test_fold, scheme = config.pipeline.encoding_scheme, threshold=100)

# data, extracted_data, split_datasets, encoder = create_dataset(data_fold=fold,
#                                                                data_config=data_config,
#                                                                preprocess_config=preprocess_config,
#                                                                encoder=encoder,
#                                                                cache=True,
#                                                                cache_image_dir=config.run_dirs.cache_dir,
#                                                                seed=config.misc.seed)

# report = classification_report(y, y_hat, target_names=text_labels, output_dict=True)

# split_datasets['train'].class_distribution

# from typing import Dict

# def calc_class_weights(class_distribution: Dict[str,int], num_samples: int):
#     num_classes = len(class_distribution)
#     weights = (1/num_classes)*(num_samples/np.array(list(class_distribution.values())))
    
#     return weights


# train_dataset = split_datasets['train']

# w = calc_class_weights(class_distribution=train_dataset.class_distribution, num_samples=train_dataset.num_samples)

# w

# train_dataset = split_datasets['train']
# class_distribution = train_dataset.class_distribution
# # class_distribution.pop('Fabaceae')

# def calc_class_weights(class_distribution: Dict[str,int], num_samples: int, encoder=None, use_int_keys=True):

#     mandatory = encoder.classes
#     class_names = list(class_distribution.keys())[:16]
#     absent = set(mandatory).union(set(class_names)) - set(class_names).intersection(mandatory)

#     full_distribution = {**class_distribution, **{class_i:0 for class_i in absent}}
#     full_distribution = {k:full_distribution[k] for k in sorted(full_distribution)}
#     if use_int_keys:
#         full_distribution = {encoder.encode([l])[0]:count for l, count in full_distribution.items()}

    
    
#     num_classes = len(full_distribution)
#     weights = (1/num_classes)*(num_samples/np.array(list(full_distribution.values())))
#     return weights

# use_int_keys=True
# mandatory = encoder.classes
# class_names = list(class_distribution.keys())[:16]
# absent = set(mandatory).union(set(class_names)) - set(class_names).intersection(mandatory)

# full_distribution = {**class_distribution, **{class_i:0 for class_i in absent}}
# full_distribution = {k:full_distribution[k] for k in sorted(full_distribution)}
# if use_int_keys:
#     full_distribution = {encoder.encode([l])[0]:count for l, count in full_distribution.items()}


# pprint([(k,full_distribution[k]) for k in sorted(full_distribution)])

# {k:full_distribution[k] for k in sorted(full_distribution)}

# help(sorted)

# for k, v in full_distribution.items():
#     print(k, v)

# mandatory
# absent

# calc_class_weights(class_distribution=train_dataset.class_distribution, num_samples=train_dataset.num_samples, include_classes)

# class_distribution

# split_datasets['train'].class_distribution

# import pandas as pd


# report_pd = pd.DataFrame(report)

# report_pd


# num_test_samples = split_datasets['test'].num_samples
# steps = num_test_samples//test_data_config.training.batch_size
# x = data['test']
# y = x.map(lambda x,y: y).take(steps).unbatch().batch(num_test_samples)



# y_collect = []

# for i, y_i in enumerate(y):
#     print(i)
#     y_collect.append(y_i)
    
#     if i>4000:
#         print('ending early')
#         break
    
# print(len(y_collect), y_collect[0])

# import numpy as np
# y_np = np.vstack(y_collect[0])
# # y_collect[:5]

# y_np.sum(axis=0)

# import dataclass


# @dataclass
# class DataExtractConfig:

#     fold_id: null
#     fold_dir: null
#     dataset_name: null
#     num_classes: null
#     threshold: 0
#     color_mode: 'grayscale'
#     num_channels: 3
#     val_split: 0.1
#     include_classes: []
#     exclude_classes: ['notcataloged','notcatalogued', 'II. IDs, families uncertain', 'Unidentified']



# @dataclass
# class DatasetConfig:
    
# #This should be added in the defaults of the global config
# params:


#   training:
#     augmentations:
#       - flip: 1.0
#     target_size: [512,512]
#     batch_size: 16
#     buffer_size: 200
#     num_epochs: 150
#     steps_per_epoch: null
#     validation_steps: null
#     use_tfrecords: false
#     samples_per_shard: 300

# # pprint(json.dumps(OmegaConf.to_container(config)))

# # config = main()#config_name="2stage_simplified_config")

# config = main(config_name="demo_simplified")

# pprint(OmegaConf.to_container(config))